I was unable to find relevant data set , so I have decided to make synthetic data set . I think instead of working on irrelevant dataset , it's good to use synthetic dataset

In [1]:
import pandas as pd
import numpy as np


In [2]:
# Generating time stamp , 1 min time stamp for 7 data 
timestamps = pd.date_range(start="2023-01-01", periods=7*24*60, freq="min")


In [3]:
# step count (low activity)
steps = np.random.poisson(lam=1, size=len(timestamps))  # here lam is average steps per minute

for hour in [7, 18]:  # 7 AM and 6 PM
    start_idx = hour * 60
    steps[start_idx : start_idx + 60] += np.random.poisson(lam=10, size=60)  # High activity

# Pet is more active in day time and less active at night

In [4]:
# Based on the threshold of steps , adding labels on the data
activity_type = np.where(
    steps == 0, "resting",
    np.where( steps < 5, "light_activity", np.where( steps < 15, "walking", "running" ) )
)

In [5]:
# Heart rate have direct correlation on the activity type
heart_rate = np.where(
    activity_type == "resting", np.random.randint(60, 80),
    np.where(
        activity_type == "walking", np.random.randint(80, 100),
        np.random.randint(100, 120)  # running
    )
)

In [6]:
df = pd.DataFrame({
    "timestamp": timestamps,
    "steps": steps,
    "activity_type": activity_type,
    "heart_rate": heart_rate,
})

In [7]:
df.head()

,timestamp,steps,activity_type,heart_rate
0,2023-01-01 00:00:00,1,light_activity,107
1,2023-01-01 00:01:00,2,light_activity,107
2,2023-01-01 00:02:00,1,light_activity,107
3,2023-01-01 00:03:00,1,light_activity,107
4,2023-01-01 00:04:00,0,resting,75


In [8]:
#Adding realistic noise to data 

# Add random spikes to steps (5% probability)
spike_prob = 0.05
spikes = np.random.choice([0, 1], size=len(df), p=[1-spike_prob, spike_prob])
df["steps"] += spikes * np.random.randint(1, 10, size=len(df))



In [9]:
# Add missing data (2% of rows)
missing_mask = np.random.rand(len(df)) < 0.02
df.loc[missing_mask, "steps"] = np.nan

# Add Gaussian noise to heart rate (±5 bpm)
hr_noise = np.random.normal(0, 2, len(df))  # Mean=0, Std=2
df["heart_rate"] = np.clip(df["heart_rate"] + hr_noise, 60, 120).astype(int)



In [10]:
# Save to CSV
df.to_csv("synthetic_pet_activity_with_noise.csv", index=False)